# Opis
Warsztat ma na celu utrwalenie wiadomości dotyczących API oraz pracy z nimi. 

Naszym zadaniem będzie pobranie danych o albumach oraz utworach dla wybranego artysty. Następnie dane te powinny zostać wyświetlone w formie raportu.

Za pomocą funkcji `input` zapytamy użytkownika o nazwę zespołu, dla którego chce pobrać dane. W dalszej kolejności:
1. pobierzemy dostępne dla podanego artysty albumy,
1. dla pobranych albumów pobierzemy listę dostępnych piosenek,
1. pobierzemy dane o tych piosenkach,
1. stworzymy raport zawierający pobrane informacje (dokładna specyfikacja poniżej).

W celu wykonania kroków 1-3 posłużymy się dedykowanym API, stworzonym na potrzeby tego zadania, którego dokumentacja jest dostępna tutaj: [klik](https://api-pad.coderslab.com/docs). API jest dostępne pod adresem: https://api-pad.coderslab.com/api, w celu autentykacji posłuż się tym kluczem: `Ft85HgGtY6Rt`.

> To zadanie można wykonać w jednej, wielokrotnie zagnieżdzonej pętli, jednak odradzamy to podejście ze względu na jego czytelność. Postaraj się dla każdego podpunktu stworzyć dedykowaną funkcję by na końcu w prosty i czytelny sposób wygenerować żądany rezultat.

### Uwaga

> Pewne endpoiny, wymagają przekazania słownika w `body`, przeczytaj ten [wątek](https://stackoverflow.com/questions/9733638/how-to-post-json-data-with-python-requests), aby dowiedzieć się w jaki sposób przekazywać słownik do endpoint'a.

## Dokumentacja API
Dokumentacja API jest stworzona przy użyciu [swagger](https://swagger.io/), przed rozpoczęciem kodowania, wspólnie (razem z wykładowcą) omówimy poszczególne jego elementy takie jak: endpointy, metody, autentykacja itp.

Na stronie dokumentacji znajdziesz również podstawowe informacje jak należy się nią posługiwać.

## Raport
Dla podanego artysty powinniśmy otrzymać raport, który wyświeli nam informacje o jego albumach oraz piosenkach, które na nim się znajdują. Powinien być on stworzony w formie katalogu. Przykładowo, jeśli wyszukiwaliśmy informacji dla `metallica`, raport powinien wyglądać następująco:
```
Artysta: metallica
|___ Album: Master Of Puppets
    |___ Tracklista:
          1. battery (remastered): Odtworzeń: 393621 | Słuchaczy: 62346 | Czas trwania: 0 | Tematyka: sadnes
          .... 
```
Wystarczy, że taki raport wyświetlimy na końcu działania naszego skryptu.

## Raport w formie pliku `csv` (opcjonalnie)
Na podstawie otrzymanych danych chcemy dodatkowo stworzyć oraz zapisać plik `csv`, o następujących wymaganiach:
- kodowanie `UTF-8`,
- separator `|`,
- nazwa pliku wynikowego: `{artysta}.csv`
- nagłówek pliku: `"artist", "album", "track", "listeners", "likes", "duration", "topic"`

## Pobranie całej dostępnej bazy (opcjonalne)
W materiałach dostępny jest plik `artists.csv`, którego zawartością jest lista artystów dostępna w API. Zmodyfikuj stworzony skrypt tak, wczytał ten plik, a następnie dla każdego artysty pobrał dostępne dla niego dane i stworzył dedykowany plik `csv`.

Rozwiązanie to powinno bazować na tym, które uzyskasz przy pracy z pojedyńczym artystą.

## Wskazówka

Przy generowaniu raportu, który ma wyświelić się w notatniku możesz posłużyć się następującym szablonem dla utworu:
```python
msg_template = ("{nb}. {track_name}: " 
                "Odtworzeń: {playcount} | "
                "Słuchaczy: {listeners} | "
                "Czas trwania: {duration} | "
                "Tematyka: {topic}"
                )
```
Który możesz uzupełnić używając metody `msg_template.format(...)`


# Rozwiązanie

> W tym miejscu podkreślamy, że każde rozwiązanie, które spełnia warunki zadania jest poprawne, to prezentowane tutaj jest tylko przykładowe. Ma ono również na celu pokazania sposobu budowania aplikacji pythonowych opartych bezpośrednio na połączeniach z zewnętrznymi serwisami.

Do rozwiązania podejdziemy następująco: 
1. Użyjemy jednego konkretnego przykładu i na tej podstawie stworzymy _szkic_ rozwiązania,
1. Na podstawie poprzedniego punktu stworzymy docelowe rozwiązanie.

## Konfiguracja części wspólnych rozwiązania

Niezależnie od etapu, potrzebujemy przygtować sobie następujące elementy:
- zaimportować wymagane biblioteki,
- przygotować sobie stałe `constants`, takie jak adres API, klucz itp

### Import bibliotek
Ponieważ będziemy wysyłać zapytania do API, na pewno potrzebujemy `requests`, dodatkowo na stronie z dokumentacją widnieje informacja, że jesteśmy ograniczeni aby wykonać 1000 żądań/min, do tego przyda nam się metoda `sleep` z biblioteki `time`:

In [1]:
import requests
from time import sleep

### Stałe

W dalszej kolejności zdefiniujemy stałe:
- `API_KEY`- klucz dostępny w treści zadania,
- `API_URL`- adres naszego api,
- `SLEEP` - będzie to wartość ile nasz kod musi odczekać po wykonaniu żądania w sekundach. Dla bezpieczeństwa ustalimy w tym notatniku, że maksymalnie w ciągu minuty będziemy odpytywać API 950 razy.

Dokumentacja informuje, że autentykacja odbywa poprzez podanie parametru `authorization`. To sobie również zapiszemy i będziemy reużywali w odpowiednich momentach (stała `AUTH`).


In [2]:
API_KEY = "Ft85HgGtY6Rt"
API_URL = "https://api-pad.coderslab.com/api"

AUTH = {'authorization': API_KEY}

SLEEP = 60/950  # czas pomiędzy kolejnymi żądaniami w sekundach

## Szkic rozwiązania

Ta część rozwiązania skupia się na stworzeniu kodu, który będzie stanowił bazę do dalszej pracy. Będziemy wykonywać zapytania na bazie konkretnych przykładów. Ćwiczenia, które wykonamy w tym celu to:
1. Na podstawie podanej nazwy artysty pobierzemy jego ID,
1. Pobierzemy listę dostępnych albumów, z niego wybierzemy jeden,
1. Dla wybranego albumy pobierzemy listę utworów, tutaj ponownie wybierzemy jeden przykład,
1. Finalnie pobierzemy dane dla wybranego utworu.
Da to nam rozeznanie w sposobie działania API oraz ułatwi kolejne kroki.

W treści zadania widnieje informacja, że dane od użytkownika powinniśmy pobrać za pomocą funkcji `input`, w tym momencie ten element pominiemy - zdefiniujemy szukanego artystę. Wartość tą będziemy przechowywać w zmiennej `artist`.

In [3]:
artist = 'metallica'

### Pobranie dostępnych albumów

Zgodnie z wymaganiami zadanie mamy zacząć od pobrania albumów dostępnych dla danego artysty. W dokumentacji możemy odnaleźć, że takie dane możemy pobrać z endpointa `artist/albums`.

Jednak jest tutaj pewien problem, ponieważ zapytania do endpointa `arists/albums` wymagają podania `artist_id`, którego w tym momencie nie posiadamy - póki co podaliśmy tylko nazwę zespołu.

Aby otrzymać id artysty (`artist_id`) musimy użyć innego endpointa: `artist/find`, który wymaga w adresie podania jego nazwy tj. `artists/find/metallica` (trzymając się naszego wyjściowego artysty).

Sprawdźmy - na początek wyślemy żądanie do API: 

In [4]:
r = requests.get(
    f"{API_URL}/artist/find/{artist}", 
    params=AUTH,
)

print(f"Status żądania: {r.status_code}")
print(f"Odpowiedź: {r.text}")

Status żądania: 200
Odpowiedź: {"artist_id":4588}


Otrzymaliśmy poprawną odpowiedź oraz informację zwrotną w postaci słownika, aby otrzymać szukane `artist_id`, musimy użyć metody `json()` a następny wyciągnąć wartość dla klucza `artist_id`:

In [5]:
artist_id = r.json()['artist_id']

Dla przykładu jeszcze wyślemy zapytanie o ID dla nieistniejącego artysty - powiedzmy, `coderslaber`. Za dokumentacją możemy powiedzieć, że oczekujemy, że api zwróci nam `status_code==200`:

In [6]:
r = requests.get(
    f"{API_URL}/artist/find/coderslaber", 
    params=AUTH,
)

print(f"Status żądania: {r.status_code}")

Status żądania: 204


Wszystko się zgadza, możemy przejść do wyjściowego zadania, czyli pobrania albumów. Po zapoznaniu się z dokumentacją stwierdzimy, że tutaj `artist_id` jest przekazywane jako adres. Wystarczy zatem drobna modyfikacja poprzedniego kodu:

In [7]:
r = requests.get(
    f"{API_URL}/artist/albums/{artist_id}",
    params=AUTH,
)

print(f"Status żądania: {r.status_code}")
print(f"Odpowiedź: {r.text}")

Status żądania: 200
Odpowiedź: [{"artist_name":"metallica","album_name":"Master of Puppets (Remastered Deluxe Box Set)","album_id":14991,"artist_id":4588},{"artist_name":"metallica","album_name":"Metallica","album_id":15193,"artist_id":4588},{"artist_name":"metallica","album_name":"Greatest Hits","album_id":15196,"artist_id":4588},{"artist_name":"metallica","album_name":"Load","album_id":15355,"artist_id":4588},{"artist_name":"metallica","album_name":"17 Years In The Life Of Metallica","album_id":15357,"artist_id":4588},{"artist_name":"metallica","album_name":"Acoustic Metal","album_id":15384,"artist_id":4588},{"artist_name":"metallica","album_name":"Reload","album_id":15397,"artist_id":4588},{"artist_name":"metallica","album_name":"Garage, Inc.","album_id":15414,"artist_id":4588},{"artist_name":"metallica","album_name":"Master of Puppets","album_id":15454,"artist_id":4588},{"artist_name":"metallica","album_name":"...and Justice for All","album_id":15466,"artist_id":4588},{"artist_name

Na podstawie powyższych wyników, widzimy, że otrzymaliśmy ponownie listę albumów zawierającą id oraz powiązaną nazwę jest ona w formacie `json`, więc analogicznie jak poprzednio:

In [8]:
albums = r.json()

Wyświetlmy dodatkowo otrzymane dane:

In [9]:
for album in albums:
    print(album)

{'artist_name': 'metallica', 'album_name': 'Master of Puppets (Remastered Deluxe Box Set)', 'album_id': 14991, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': 'Metallica', 'album_id': 15193, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': 'Greatest Hits', 'album_id': 15196, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': 'Load', 'album_id': 15355, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': '17 Years In The Life Of Metallica', 'album_id': 15357, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': 'Acoustic Metal', 'album_id': 15384, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': 'Reload', 'album_id': 15397, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': 'Garage, Inc.', 'album_id': 15414, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': 'Master of Puppets', 'album_id': 15454, 'artist_id': 4588}
{'artist_name': 'metallica', 'album_name': '...and Justice for All', 'album_id': 1

Co kończy zadanie 1.

## Pobranie listy utworów

Ta część już jest o wiele prostsza. Ponieważ mamy już do dyspozycji `album_id`, wystarczy tylko w dokumentacji odszukać endpoint, który za to zadanie odpowiada- `/album/tracks`.

Isotnym do zaznaczenia jest fakt, że tutaj dane przejazujemy w formie `body` co więcej muszą być one w formacie `json`. Po przeczywaniu wątku podanego we wskazówce: [click](https://stackoverflow.com/questions/9733638/how-to-post-json-data-with-python-requests), wiemy, że należy te informacje przekazać za pomocą parametru `json=...`.

Ponieważ `albums` zawiera już kilka elementów, wyizolujmy sobie przykład i na jego podstawie opracujmy żądanie:

In [10]:
album = albums[0] # izolacja pojedyńczego przypadku
print(f"Dane albumu: {album}")

Dane albumu: {'artist_name': 'metallica', 'album_name': 'Master of Puppets (Remastered Deluxe Box Set)', 'album_id': 14991, 'artist_id': 4588}


In [11]:
album_id = album['album_id']
r = requests.post(
    f"{API_URL}/album/tracks",
    params=AUTH,
    json={'album_id': album_id},
)

print(f"Status żądania: {r.status_code}")
print(f"Odpowiedź: {r.text}") # wyświetli nam ID utworów, które znajdują się na albumie - w tym przypadku jest to jednoelementowa lista

Status żądania: 200
Odpowiedź: [74813]


W odpowiedzi otrzymaliśmy kolejną listę, tym razem z `id utworów`. Kończy to pracę z tą częścią zadania.

In [12]:
tracks = r.json()

## Pobranie danych o utworze

Mamy już wszystkie potrzebne informacje, aby pobrać dane dla wybranego utworu. W tym celu należy zapoznać się z dokumentacją endpointa `track/get`.

W tym przypadku rozwiązanie jest analogiczne do pobierania danych o liście utworów- wystarczy, że zmienimy adres endpointa i dopowiednio przekażemy `track_id` w `body`.

Jak wcześniej, wyizolujemy sobie jeden wybrany utwór i na jego przykładzie zbudujemy żądanie:

In [13]:
track_id = tracks[0]
print(f"Id utworu: {track_id}")

Id utworu: 74813


teraz żądanie:

In [14]:
r = requests.post(
    f"{API_URL}/track/get",
    params=AUTH,
    json={'track_id': track_id}
)
print(f"Status żądania: {r.status_code}")
print(f"Odpowiedź: {r.text}")

Status żądania: 200
Odpowiedź: {"id":74813,"artist_name":"metallica","track_name":"battery (remastered)","lyrics":"lash action return reaction weak rip tear away hypnotize power crush cower battery stay smash boundaries lunacy stop battery pound aggression turn obsession kill battery kill family battery battery battery lyric commercial","release_date":"1986","len":32,"age":0.48571428571428504,"topic":"sadness","album_name":"Master of Puppets (Remastered Deluxe Box Set)","playcount":393621,"listeners":62346,"duration":0,"album_id":14991,"artist_id":4588}


Otrzymujemy zatem ponownie słownik z danymi. Tym razem jednak otrzymujemy dane o utworze i nie musimy niczego dalej pobierać, znajdziemy tutaj informacje o:

- nazwie utworu - `track_name`,
- liczbie odtworzeń - `playcount`,
- liczbie słuchaczy - `listeners`,
- wieku utworzy - `age`,
- czasie trwania - `duration`,
- tematyce - `topic`.

Co kończy tę część warsztatu.

### Stworzenie raportu

W tym momencie pominiemy stworzenie raportu, ponieważ szkic dotyczył pojedyńczych przykładów dla żądań.

## Podsumowanie

W tej cześci pobraliśmy dane zgodnie z wymaganiami zadania. W każdym przypadku wybraliśmy sobie jeden przykład albumu/utworu, na podstawie których opracowaliśmy kształt poszczególnych żądań.

Dalej skupimy się przede wszystkim na pełnym rozwiązaniu warsztatu oraz przygotowaniu procedur, które ułatwią wykonanie całego kodu.

# Docelowe rozwiązanie

W tej części dopracujemy nasze rozwiązanie, kroki, które wcześniej wykonaliśmy opakujemy w postać funkcji, dzięki czemu finalna postać kodu będzie zgodna z dobrymi praktykami. Bazując na budowie API, stworzymy metody takie jak:
- bazujące bezpośrednio na API:
    - `get_artist_id`, która na podstawie nazwy artysty pobieże jego `id`,
    - `get_artist_albums`, zwracająca albumy powiązane z danym artystą,
    - `get_album_tracks` - pobierająca listę utworów z danego albumu,
    - `get_track_data` - pobierająca dane utworu,
- metody wyższego poziomu, łączące powyższe:
    - `download_album_tracks` - wykorzystanie `get_album_tracks` oraz `get_track_data`, ze względu na strukturę zwracanych przez nie wyników,
    - `download_artist_albums` - jako połączenie `get_artist_albums` oraz `download_album_tracks`.

Istotną różnicą będzie to, że musimy w tym momencie obsłużyć obsługę wielu rekordów - przypomnijmy, że wcześniej posługiwaliśmy się pojedyńczymi rekordami. To obsłużymy używając pętli `for`, gdzie potrzebne.

## Uogólnienie szkicu

W tej części zdefiniujemy wcześniej określone metody. Implementację zaczniemy od tych _niskopoziomowych_, czyli tych, które bazują na połączeniu z API, więc będziemy używać kodu, który został stworzony w trakcie tworzenia szkicu. Po wykonaniu tej części przejdziemy to metod _wyższego_ poziomu.

### `get_artist_id`

Aby poprawnie zdefiniować metodę musimy upewnić się, że poprawnie obsłużymy `status_code`:
- w przypadku `200` lub `204`, wystarczy, że zwrócimy wartość `artist_id`
- innym przypadku zwrócimy informację o nieobsługiwanym `status_code`, przykładowo: `Unhandled status code: {status_code}`

In [15]:
def get_artist_id(artist):
    r = requests.get(
        f"{API_URL}/artist/find/{artist}", 
        params=AUTH,
    )
    sleep(SLEEP) # czekamy ustaloną wartość czasu po wykonaniu żądania

    if r.status_code in (200, 204):
        result = r.json()['artist_id']
        return result
    print(f"Unhandled status code: {r.status_code}")

### `get_artist_albums`
Podobnie jak wcześniej potrzebujemy dodać obsługę `status_code`, zrobimy to analogicznie jak wcześniej. Istotną różnicą pomiędzy tym rozwiązaniem będzie obsłużenie wielu rekordów. Ze względu na późniejszą konieczność używania tych danych w różny sposób- czy to w innych API czy potem do raportu, w tym rozwiązaniu wynikiem metody będzie następująca struktura:
```python
{0: 'Album A',
 1: 'Album B',
 2: 'Album C'
}
```
Czyli będzie to po prostu słownik, którego kluczem będzie `id` albumu a wartością jego nazwa. Musimy zatem dodać następujące kroki:
- stworzyć słownik, który będzie przechowywał mapowanie klucza albumu na jego nazwę - `albums`,
- z wyniku żądania wyciągnąć `album_id` oraz `album_name` dla każdego albumu,
- tak uzyskane elementy dodać do zmiennej `albums`.

> Dlaczego chcemy otrzymaną tablicę przekształcić w słownik? W ten sposó otrzumujemy prosty sposób na odwołwanie się do poszczególnego albumu i pobrania potrzebnej informacji. Gdybyśmy zdecydowali się na zwrócenie wartości wprost z API czyli w formie tablicy z zagnieżdżonymi słownikami wymagałoby to w dalszych krokach filtrowania, co mogłoby skomplikować dodatkowo rozwiązanie.


In [16]:
def get_artist_albums(artist_id):
    r = requests.get(
        f"{API_URL}/artist/albums/{artist_id}",
        params=AUTH,
    )
    sleep(SLEEP) # czekamy ustaloną wartość czasu po wykonaniu żądania

    if r.status_code not in (200, 204):
        print(f"Unhandled status code: {r.status_code}")
        return []

    records = r.json()

    albums = {}  # definiujemy pusty słownik, jako kontener na dane wynikowe
    for record in records:  # iterujemy po wszystkich dostępnych albumach
        album_id = record['album_id']
        album_name = record['album_name']
        albums[album_id] = album_name  # dodajemy nowy album do zbioru
    return albums

### `get_album_tracks`

Ta funkcja ma na celu pobranie utworów dla danego albumu. W tej części nie ma potrzeby abyśmy dodatkowo modyfikowali dane wynikowe- mogą zostać zwrócone prosto z API.

In [17]:
def get_album_tracks(album_id):
    r = requests.post(
        f"{API_URL}/album/tracks",
        params=AUTH,
        json={'album_id': album_id},
    )
    sleep(SLEEP)

    if r.status_code not in (200, 204):
        print(f"Unhandled status code: {r.status_code}")
        return []

    tracks_id = r.json()
    return tracks_id

### `get_track_data`

Podobnie jak w poprzednim podpunkcie, te funkcja będzie tylko nakładką na samo żądanie:

In [18]:
def get_track_data(track_id):
    r = requests.post(
        f"{API_URL}/track/get",
        params=AUTH,
        json={'track_id': track_id}
    )
    sleep(SLEEP)

    if r.status_code not in (200, 204):
        print(f"Unhandled status code: {r.status_code}")
        return []
        
    result = r.json()
    return result    

### `download_album_tracks`

Zanim przejdziemy do zdefiniowania funkcji `download_album_tracks`, podsumujmy co mamy do tej pory:
1. pobranie danych o artyście,
1. pobranie dostępnych albumów artysty,
1. pobranie listy dostępnych utwórów na albumie,
1. pobranie danych pojedyńczego utworu.

Brakującym ogniwem jest tutaj połączenie kroków 3. i 4. w taki sposób, aby pobrać dane wszystkich utworów dostępnych na albumie - i za to właśnie będzie odpowiadała funkcja `download_album_tracks`.

In [19]:
def download_album_tracks(album_id):
    tracks_id = get_album_tracks(album_id)
    tracks = []

    for track_id in tracks_id:
        data = get_track_data(track_id)
        tracks.append(data)
    return tracks

### `download_artist_albums`

Jak już wcześniej wspomnieliśmy, zadaniem tej funkcji będzie po prostu pobranie danych albumów podanego artysty. Musi ona zatem implementować następującą logikę:
- jako parametr przyjomować `artist_id`,
- pobrać albumy, które są dla niego dostępne - tutaj wykorzystamy `get_artist_albums`,
- pobrać dane o znajdujące się na tym albumie - tutaj możemy posłużyć się już wcześniej zdefiniowaną `download_album_tracks`.

Co warto w tym miejscu zaznaczyć, to fakt, że z `get_artist_albums`, otrzymamy słownik. Będziemy chcieli utrzymać tą konwencję i proponujemy, aby ta metoda zwracała dwie zmienne:
- `albums`- czyli wynik `get_artist_albums`, ponieważ zawiera mapowanie id do nazwy, co będzie potrzebne do wygenerowania raportu,
- `albums_tracks` - czyli listę utworów przypisaną do danego albumu, stukturą będzie podobne do struktury z metody `get_artist_albums`:
```python
{1: [{dane utworu}]}
```
z tą różnicą, że jako wartości przetrzymywana będzie lista z danymi o utworze.

In [20]:
def download_artist_albums(artist_id):
    albums = get_artist_albums(artist_id)

    albums_tracks = {}
    for id, album in enumerate(albums, 1):
        print(f"Pobieram {id} z {len(albums)}")
        tracks = download_album_tracks(album)
        albums_tracks[album] = tracks
    return albums, albums_tracks

## Pobranie danych dla wybranego zespołu

W tym miejscu stworzymy miejscie, w którym wykonamy całość stworzonego kodu. Dodamy również brakujący do tej pory element, czyli pobranie interesującej nas nazwy zespołu przy pomocy `input`.

In [21]:
artist = input("Podaj nazwę zespołu")
print(f"Podano zespół: {artist}")

Podano zespół: madonna


In [22]:
artist_id = get_artist_id(artist)
albums, tracks = download_artist_albums(artist_id)

Pobieram 1 z 10
Pobieram 2 z 10
Pobieram 3 z 10
Pobieram 4 z 10
Pobieram 5 z 10
Pobieram 6 z 10
Pobieram 7 z 10
Pobieram 8 z 10
Pobieram 9 z 10
Pobieram 10 z 10


### Podglądnięcie danych

In [23]:
albums

{1476: 'Madonna',
 1830: 'Like a Virgin',
 1838: 'Frozen',
 1913: 'True Blue',
 2138: 'Like a Prayer',
 2209: 'The Immaculate Collection',
 2355: 'The Girlie Show: Live Down Under',
 2532: 'Bedtime Stories',
 2773: 'Ray of Light',
 3287: 'The Confessions Tour'}

In [24]:
tracks

{1476: [{'id': 7386,
   'artist_name': 'madonna',
   'track_name': 'holiday',
   'lyrics': 'holiday celebrate holiday celebrate take holiday take time celebrate life nice everybody spread word gonna celebration world nation time good time forget time yeah come release pressure need holiday take holiday take time celebrate life nice turn world bring days trouble time celebrate shine come things better need holiday take holiday take time celebrate life nice holiday celebrate holiday celebrate take holiday take time celebrate life nice holiday celebrate holiday celebrate come nation',
   'release_date': '1979',
   'len': 73,
   'age': 0.5857142857142851,
   'topic': 'world/life',
   'album_name': 'Madonna',
   'playcount': 2310260,
   'listeners': 444332,
   'duration': 241000,
   'album_id': 1476,
   'artist_id': 427},
  {'id': 8689,
   'artist_name': 'madonna',
   'track_name': 'lucky star',
   'lyrics': 'cause shine think start glow need light baby know starlight starbright tonight sta

## Stworzenie raportów

Ostatnią częścią warsztatu jest odpowiednie wyświetlenie oraz zapisanie danych.

### Wyświetlenie danych w notatniku

Na sam początek zajmiemy się wyświetleniem danych w notatniku, dlatego tutaj sobie skopiujemy podany we wskazówce kod.

In [25]:
msg_template = ("{nb}. {track_name}: " 
                "Odtworzeń: {playcount} | "
                "Słuchaczy: {listeners} | "
                "Czas trwania: {duration} | "
                "Tematyka: {topic}"
       )

to co teraz nam pozostaje to wygenerować teraz raport, przypomnijmy w następującym formacie:
```
Artysta: metallica
|___ Album: Master Of Puppets
    |___ Tracklista:
          1. battery (remastered): Odtworzeń: 393621 | Słuchaczy: 62346 | Wiek: 0.48571428571428504 | Czas trwania: 0 | Tematyka: sadnes
          .... 
```
Na podstawie powyższego szablonu oraz danych, które pobraliśmy możemy określić dalszy zakres prac:
1. Artysta ma zostać wyświetlony tylko raz,
1. Dla każdego albumu potrzebujemy wyświetlić jego nazwę oraz tracklistę,
1. Dla każdego utworu z tracklisty, uzupełniamy szablon zgodnie z wytycznymi.

Przekładając to na język python, do napisania mamy dwie pętle:
- jedna, która będzie iterowała po albumach wykonawcy
- kolejna, która będzie iterowała po utworach z danego albumu.

Kodując powyższe otrzymamy:

In [26]:
print(f"Artysta: {artist}")  # wyświetlenie podanego artysty

# iterujemy po albumach dla danego wykonawcy
for album_id in albums:
    print(f"|___ Album: {albums[album_id]}")  # wyświetlamy informacje o albumie
    print(f"    | Tracklista:")  # wyświetlamy nagłówek (4 spacje)

    album_tracks = tracks[album_id]  # pobieramy utwory znajdujące się na albumie
    for nb, track in enumerate(album_tracks, 1):  # przebiegamy po kolejnych utorach z albumy

        # odpowiednio filtrujemy szablon wyjściowy
        print("     " + msg_template.format(  # dodatkowo dodaliśmy 6 spacji, aby ładnie się lista ułożyła
            nb=nb,
            track_name=track['track_name'],
            playcount=track['playcount'],
            listeners=track['listeners'],
            duration=track['duration'],
            topic=track['topic'],
            ))

Artysta: madonna
|___ Album: Madonna
    | Tracklista:
     1. holiday: Odtworzeń: 2310260 | Słuchaczy: 444332 | Czas trwania: 241000 | Tematyka: world/life
     2. lucky star: Odtworzeń: 1551916 | Słuchaczy: 296486 | Czas trwania: 215000 | Tematyka: night/time
|___ Album: Like a Virgin
    | Tracklista:
     1. material girl: Odtworzeń: 4414829 | Słuchaczy: 805512 | Czas trwania: 219000 | Tematyka: world/life
     2. stay: Odtworzeń: 200731 | Słuchaczy: 45311 | Czas trwania: 0 | Tematyka: sadness
|___ Album: Frozen
    | Tracklista:
     1. love don't live here anymore: Odtworzeń: 305977 | Słuchaczy: 62854 | Czas trwania: 285000 | Tematyka: world/life
|___ Album: True Blue
    | Tracklista:
     1. love makes the world go round: Odtworzeń: 202318 | Słuchaczy: 46877 | Czas trwania: 276000 | Tematyka: world/life
     2. jimmy jimmy: Odtworzeń: 207321 | Słuchaczy: 49268 | Czas trwania: 236000 | Tematyka: sadness
     3. open your heart: Odtworzeń: 1842486 | Słuchaczy: 257483 | Czas trwan

### Zapisanie danych do pliku CSV

Aby zapisać dane do pliku `csv` przyda się nam dodatkowo dedykowana do tego celu biblioteka pythonowa, która była omawiana w trakcie pierwszego zjazdu:

In [27]:
import csv

Pomimo innego formatu wyjściowego, szkielet rozwiązania będzie podobny- stworzymy dwie pętle, które będą generować kolejne wiersze do pliku `csv`. 

Musimy jednak zadbać dodatkowo o stworzenie nagłówka, przed dodawaniem wierszy.

In [28]:
file_name = f"{artist}.csv"  # stworzenie nazwy pliku
f = open(file_name, mode='w+', encoding="UTF-8", newline='')  # ustawienie kodowania, trybu pracy z plikiem

header = ["artist", "album", "track", "listeners", "likes", "duration", "topic"]  # kopiujemy z treści zadania postać nagłówka

# definiujemy plik csv zgodnie z resztą wymagań
writer = csv.writer(
    f, 
    delimiter="|", 
    quotechar='"',  # dzięki temu znaki będą opatrzone dodatkowo "text"
    quoting=csv.QUOTE_NONNUMERIC,  # określamy, że nie chcemy aby liczby były w "123"
    doublequote=True  # w przypadku piosenek lub albumów, w których występuje " zostaną one dodatkowo opatrzone """
)
writer.writerow(header)  # dodajemy nagłówek

for album_id in albums:  # przebiegamy po poszczególnych albumach
    album_tracks = tracks[album_id]  # pobieramy utwory znajdujące się na albumie
    for id, track in enumerate(album_tracks, 1):  # tworzymy nowy wiersz zawierający wymagane dane
        writer.writerow([
            artist,
            albums[album_id],
            track['track_name'],
            track['playcount'],
            track['listeners'],
            track['duration'],
            track['topic'],
        ])
f.close()

> Ze względu na czytelność kodu (kolejny poziom wcięcia) w tym rozwiązaniu nie otwieraliśmy pliku przy użyciu kontekstu. Przy takim podejściu należy pamiętać o tym, aby na końcu pracy z plikiem użyć `f.close()`  

## Podsumowanie

W tym artykule zapoznaliśmy się z przykładowym rozwiązaniem zadania. Pracę rozpoczęliśmy od stworzenia szkicu, czyli po prostu pobrania danych z API, aby w kolejnej części dokonać uogólnienia rozwiązania i stworzyć dedykowane funkcje, takie jak:
- `get_artist_id`, która na podstawie nazwy artysty pobieże jego `id`,
- `get_artist_albums`, zwracająca albumy powiązane z danym artystą,
- `get_album_tracks` - pobierająca listę utworów z danego albumu,
- `get_track_data` - pobierająca dane utworu,
których zadaniem było tylko pobieranie danych z API, bez większych ingerencji w ich wynik o ile nie było to zasadne - jak w przypadku `get_artist_albums`, gdzie dokonaliśmy konwersji listy na słownik.

Bazując na powyższych zdefiniowaliśmy również metody _wyższego poziomu_, których zadaniem było odpowiednie połączenie funkcji bazujących bezpośrednio na API, co pozwoliło nam finalnie na pobranie danych za pomocą dwóch, linijek kodu:
```python
artist_id = get_artist_id(artist)
albums, tracks = download_artist_albums(artist_id)
```
Dzięki czemu rozwiązanie stało się bardzo czytelne.

Ostatnią częścią zadania było stworzenie dwóch raportów:
- jednego wyświetlanego w konsoli,
- drugiego, którego wynikiem jest plik csv o nazwie `{artist_name}.csv` w zależności od tego, dla kogo pobieraliśmy dane.